<a href="https://colab.research.google.com/github/fgotelip/CartolaFC/blob/main/Cartola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Python-CartolaFC

In [ ]:
import cartolafc
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [ ]:
api = cartolafc.Api()
mercado = api.mercado()
ultima_rodada = mercado.rodada_atual - 1

def pontuacao_capitao(time):
  for j in range(11):
    if time.atletas[j].is_capitao:
      pontuacao_capitao = time.atletas[j].pontos
  capitao = pontuacao_capitao*0.5
  return capitao

def infos_por_rodada(times,rodada):
    global api
    infos_times_rodada = []
    for time in times:
        infos_times_rodada.append((api.time(time,rodada),rodada))
    return infos_times_rodada

def maior_pontuador(infos_rodada,tem_capitao):
    pontuacoes = []
    for info in infos_rodada:
        pontuacao = {}
        pontuacao['Rodada'] = info[1]
        pontuacao['Time'] = info[0].info.nome
        ultima_pontuacao = info[0].ultima_pontuacao
        if tem_capitao == 0:
          ultima_pontuacao -= pontuacao_capitao(info[0])
        pontuacao['Pontuacao'] = ultima_pontuacao
        pontuacao['Retorno'] = 25 if info[1] in (1,2) else 27
        pontuacoes.append(pontuacao)
    return max(pontuacoes, key=lambda x: x['Pontuacao'])

def campeoes_por_rodada(times,tem_capitao):
    global api
    global ultima_rodada
    campeoes = []

    for rodada in range(1, ultima_rodada + 1):
        infos_rodada = infos_por_rodada(times,rodada)
        campeoes.append(maior_pontuador(infos_rodada,tem_capitao))
    return campeoes

In [ ]:
times = [29674391,14156535,12039729,25577506,25330326,49127596,8812795]
campeoes = campeoes_por_rodada(times,1)

In [ ]:
numero_rodadas = 38
rodadas_restantes = numero_rodadas - ultima_rodada
primeiro_lugar = 330
segundo_lugar = 160
terceiro_lugar = 80
investimento_total = 226

investimento_parcial = 10 + (ultima_rodada-2)*6


df_campeoes = pd.DataFrame(campeoes)

In [ ]:
df_vitorias = df_campeoes["Time"].value_counts().reset_index()

df_vitorias.columns = ["Time", "Vitorias"]

retorno = df_campeoes.groupby("Time")["Retorno"].sum().reset_index()

df_vitorias = df_vitorias.merge(retorno,on="Time",how="left")
df_vitorias["Lucro Atual"] = df_vitorias["Retorno"] - investimento_parcial
df_vitorias["Lucro Atual Final"] = df_vitorias["Retorno"] - investimento_total

df_vitorias["Rodadas Para Lucro"] = np.where(
    df_vitorias["Lucro Atual Final"] > 0,
    0,
    np.ceil(np.abs(df_vitorias["Lucro Atual Final"]/27)).astype(int)
)

In [ ]:
numero_rodadas = 0
parcial_final = df_vitorias[["Time","Lucro Atual Final"]]
parcial_final = parcial_final.rename(columns={"Lucro Atual Final": "Lucro"})
df_parcial_final = parcial_final.T
df_parcial_final.columns = df_parcial_final.iloc[0]
df_parcial_final = df_parcial_final[1:]
df_parcial_final["Num Rodada"] = numero_rodadas

df_times = df_parcial_final.columns.drop("Num Rodada")


ultima_parcial = df_parcial_final.iloc[-1].copy()

for parcial in range(rodadas_restantes):
  nova_parcial = ultima_parcial.copy()
  nova_parcial["Num Rodada"] += 1
  nova_parcial[df_times] += 27
  df_parcial_final = pd.concat([df_parcial_final, pd.DataFrame([nova_parcial])])
  ultima_parcial = nova_parcial

df_parcial_rodadas = df_parcial_final.copy()

df_parcial_3 = df_parcial_final.copy()
df_parcial_3[df_times] += terceiro_lugar

df_parcial_2 = df_parcial_final.copy()
df_parcial_2[df_times] += segundo_lugar

df_parcial_1 = df_parcial_final.copy()
df_parcial_1[df_times] += primeiro_lugar


In [ ]:
caminho = "campeonato.xlsx"

with pd.ExcelWriter(caminho, engine='openpyxl') as writer:
    df_campeoes.to_excel(writer, sheet_name='Campeoes', index=False)
    df_vitorias.to_excel(writer, sheet_name='Vitorias', index=False)
    df_parcial_rodadas.to_excel(writer, sheet_name='Rodadas')
    df_parcial_3.to_excel(writer, sheet_name='3º')
    df_parcial_2.to_excel(writer, sheet_name='2º')
    df_parcial_1.to_excel(writer, sheet_name='1º')

In [ ]:
def formatar_real(aba,colunas):
  for coluna in colunas:
    for cell in aba[coluna]:
      if isinstance(cell.value, (int, float)):
        cell.number_format = 'R$ #,##0'

wb = load_workbook(caminho)

campeoes = wb["Campeoes"]
vitorias = wb["Vitorias"]
parcial_rodadas = wb["Rodadas"]
parcial_3 = wb["3º"]
parcial_2 = wb["2º"]
parcial_1 = wb["1º"]

colunas_campeoes = ["D"]
colunas_vitorias = ["C","D","E"]
colunas_parciais = ["B","C","D","E","F","G","H"]

formatar_real(campeoes,colunas_campeoes)
formatar_real(vitorias,colunas_vitorias)
formatar_real(parcial_rodadas,colunas_parciais)
formatar_real(parcial_3,colunas_parciais)
formatar_real(parcial_2,colunas_parciais)
formatar_real(parcial_1,colunas_parciais)

In [ ]:
from openpyxl.worksheet.table import Table, TableStyleInfo

for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    # Create a table from the entire data range
    # Assumes the first row contains headers
    tab = Table(displayName=f"{sheet_name.replace(' ', '_')}_Table", ref=f"A1:{ws.cell(row=ws.max_row, column=ws.max_column).coordinate}")

    # Add a default style to the table
    style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                           showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    tab.tableStyleInfo = style

    # Add the table to the worksheet
    ws.add_table(tab)

In [ ]:
from openpyxl.styles import PatternFill
# Vermelho claro
red_fill = PatternFill(start_color="FFFF9999", end_color="FFFF9999", fill_type="solid")

# Verde claro
green_fill = PatternFill(start_color="FF99FF99", end_color="FF99FF99", fill_type="solid")

# Define the sheets and columns to apply conditional formatting
sheets_to_format = ['Rodadas', '3º', '2º', '1º']
columns_to_format_vitorias = ['B', 'C', 'D', 'E']

for sheet_name in sheets_to_format:
    ws = wb[sheet_name]
    columns_to_format = [chr(ord('A') + col_idx) for col_idx in range(1, ws.max_column - 1)]

    for col_letter in columns_to_format:
        for cell in ws[col_letter]:
            # Check if the cell value is a number (int or float)
            if isinstance(cell.value, (int, float)):
                if cell.value < 0:
                    cell.fill = red_fill
                elif cell.value > 0:
                    cell.fill = green_fill

In [ ]:
from openpyxl.styles import Alignment

for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter # Get the column name
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
                cell.alignment = Alignment(horizontal='center', vertical='center')
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2 # Add a little extra padding
        ws.column_dimensions[column].width = adjusted_width

wb.save(caminho)

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_writer.py:274: UserWarning: File may not be readable: column headings must be strings.
  warn("File may not be readable: column headings must be strings.")
